In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 70)
pd.set_option('display.max_rows',150)

In [3]:
df = pd.read_csv('input/bases/210729_f11_l2000.csv', sep=';', dtype='object')

In [4]:
def gr(df3, dfna):
    df = df3.copy()
    i = 0
    idf=0
    while i < dfna.shape[0]:
        i = dfna.index[idf]
        df.iloc[i-4,21] = dfna.iloc[idf-2,1:2].values[0]
        df.iloc[i-4,22:27] = dfna.iloc[idf-1,1:6].values
        df.iloc[i-4,27:49] = dfna.iloc[idf,1:23].values
        idf+=3
    return df 

In [5]:
def grdos(dfarg, dfna):
    df = dfarg.copy()
    i = 0
    idf=0
    while idf < dfna.shape[0]:
        i = dfna.index[idf]
        df.iloc[i-2,3] = df.iloc[i-1,1]
        df.iloc[i-2,4:6] = df.iloc[i,1:3].values
        df.iloc[i-2,6:21] = df.iloc[i+1,1:16].values
        idf+=1
    return df 

Limpieza parte 1 

In [6]:
dfnull = df[df.isnull().all(axis=1)]
df = df[~df.isnull().all(axis=1)].reset_index(drop=True)

Limpieza parte 2

In [7]:
dfna = df[df['Nro. F11'].isna()]
idfna = dfna.index.values 

dfnacliente = df.loc[(df['Nro. F11'].isna())&((df['F03 Asociado']=='CLIENTE')|(df['F03 Asociado']=='EMPRESA'))]
index_dfnac = dfnacliente.index

menosuno = (index_dfnac-1).isin(idfna)
menosdos = (index_dfnac-2).isin(idfna)

oklist = []
for i in range(len(index_dfnac)): 
    oklist.append((menosuno[i]==True) & (menosdos[i]==True))
    
itochange = index_dfnac[oklist]

acamb = dfnacliente.loc[itochange]

df2 = gr(df, acamb)
iuno = acamb.index.values
idos = acamb.index.values-1
itres = acamb.index.values-2
df3 = df2.loc[~df2.index.isin(iuno)]
df3 = df3.loc[~df3.index.isin(idos)]
df3 = df3.loc[~df3.index.isin(itres)]
df3.reset_index(drop=True, inplace=True)

dfuno = df.loc[(df.index.isin(iuno))|(df.index.isin(idos))|(df.index.isin(itres))]
dfuno.to_excel(f'output/df-deleted-1.xlsx', sheet_name=f'f11', index=False) 



In [8]:
dfnacliente.to_excel(f'output/dfna-2.xlsx', sheet_name=f'f11', index=False) 

Limpieza parte 2

In [9]:
dfna = df3[df3['Nro. F11'].isna()]

indexdigit = df3.loc[(df3['Nro. F11'].isna())&(df3['F03 Asociado'].str.isdigit())].index

index_dfna = dfna.index.values
masuno = (indexdigit+1).isin(index_dfna)
menosuno = (indexdigit-1).isin(index_dfna)

oklist = []
for i in range(len(indexdigit)): 
    oklist.append((masuno[i]==True) & (menosuno[i]==True))
    
itochange = indexdigit[oklist]

In [10]:
acamb = dfna.loc[itochange]
df4 = grdos(df3, acamb)
iuno = acamb.index.values-1
idos = acamb.index.values
itres = acamb.index.values+1
df4 = df4.loc[~df4.index.isin(iuno)]
df4 = df4.loc[~df4.index.isin(idos)]
df4 = df4.loc[~df4.index.isin(itres)]
df4.reset_index(drop=True, inplace=True)

dfdos = df3.loc[(df3.index.isin(iuno))|(df3.index.isin(idos))|(df3.index.isin(itres))]
dfdos.to_excel(f'output/df-deleted-2.xlsx', sheet_name=f'f11', index=False) 

Limpieza parte 3

In [11]:
bogota = df4.loc[(df4['Nro. F11'].isna())&(df4['F03 Asociado']=='BOGOTA')]

for i in bogota.index:
    df4.iloc[i-1,22:27] = df4.iloc[i,1:6].values
    
df5 = df4[~df4.index.isin(bogota.index)].reset_index(drop=True)

cliente/empresa

In [12]:
cliente_empresa = df5.loc[(df5['Nro. F11'].isna())&((df5['F03 Asociado']=='CLIENTE')|(df5['F03 Asociado']=='EMPRESA'))]

for i in cliente_empresa.index:
    df5.iloc[i-1,27:49] = df5.iloc[i,1:23].values
    
df6 = df5[~df5.index.isin(cliente_empresa.index)].reset_index(drop=True)

direccion

In [13]:
direc = df6.loc[(~df6['F03 Asociado'].str.isdigit().fillna(False))&(df6['Nro. F11'].isna())&(df6['F03 Asociado']!='CD FALABELLA')]

for i in direc.index:
    df6.iloc[i-1,21:27] = df6.iloc[i,1:7].values
    df6.iloc[i-1,27:49] = df6.iloc[i,27:49].values
    
df7 = df6[~df6.index.isin(direc.index)].reset_index(drop=True)

Guardar archivos 

In [14]:
df3.to_excel(f'output/df3.xlsx', sheet_name=f'f11', index=False) 
df4.to_excel(f'output/df4.xlsx', sheet_name=f'f11', index=False) 
df5.to_excel(f'output/df5.xlsx', sheet_name=f'f11', index=False) 
df6.to_excel(f'output/df6.xlsx', sheet_name=f'f11', index=False) 
df7.to_excel(f'output/df7.xlsx', sheet_name=f'f11', index=False) 